DAY- 8(Data Warehousing)

TOPIC: Data Warehousing Fundamentals
   1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.
   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.
   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.


In [ ]:
-- Create the fact table
CREATE TABLE Sales (
    sales_id SERIAL PRIMARY KEY,
    product_id INT,
    customer_id INT,
    date_id INT,
    sales_amount DECIMAL(10, 2),
    quantity_sold INT,
    FOREIGN KEY (product_id) REFERENCES Products (product_id),
    FOREIGN KEY (customer_id) REFERENCES Customers (customer_id),
    FOREIGN KEY (date_id) REFERENCES Time (date_id)
);

-- Populate the fact table with sample data
INSERT INTO Sales (product_id, customer_id, date_id, sales_amount, quantity_sold)
VALUES
    (1, 1, 1, 100.00, 2),
    (2, 2, 1, 150.00, 3),
    (1, 3, 2, 75.50, 1),
    -- Additional sample data...
    ;


In [ ]:
-- Retrieve total sales amount for each product
SELECT p.product_name, SUM(s.sales_amount) AS total_sales_amount
FROM Sales s
JOIN Products p ON s.product_id = p.product_id
GROUP BY p.product_name;

-- Retrieve sales amount by year and quarter
SELECT t.year, t.quarter, SUM(s.sales_amount) AS total_sales_amount
FROM Sales s
JOIN Time t ON s.date_id = t.date_id
GROUP BY t.year, t.quarter;

-- Retrieve sales amount by customer and product category for a specific month
SELECT c.customer_name, p.category, SUM(s.sales_amount) AS total_sales_amount
FROM Sales s
JOIN Customers c ON s.customer_id = c.customer_id
JOIN Products p ON s.product_id = p.product_id
JOIN Time t ON s.date_id = t.date_id
WHERE t.month = 'January'
GROUP BY c.customer_name, p.category;


TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.


In [ ]:
import pandas as pd
import psycopg2

# Extraction
def extract_data(csv_file_path):
    return pd.read_csv(csv_file_path)

# Transformation
def transform_data(data):
    # Apply business rules or calculations
    data['transformed_column'] = data['original_column'] * 2
    return data

# Loading
def load_data(data, connection):
    cursor = connection.cursor()

    # Create or truncate tables in the data warehouse
    cursor.execute("DROP TABLE IF EXISTS table_name;")
    cursor.execute("CREATE TABLE table_name (column1 datatype1, column2 datatype2);")

    # Load the transformed data into the data warehouse
    for _, row in data.iterrows():
        insert_query = "INSERT INTO table_name (column1, column2) VALUES (%s, %s);"
        values = (row['transformed_column'], row['other_column'])
        cursor.execute(insert_query, values)

    # Commit the changes and close the cursor
    connection.commit()
    cursor.close()

# Main ETL process
def etl_process(csv_file_path):
    # Extract
    extracted_data = extract_data(csv_file_path)

    # Transform
    transformed_data = transform_data(extracted_data)

    # Load
    connection = psycopg2.connect(
        host='your_host',
        port='your_port',
        database='your_database',
        user='your_username',
        password='your_password'
    )
    load_data(transformed_data, connection)
    connection.close()

# Execute the ETL process
etl_process('path_to_csv_file.csv')


TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.


In [ ]:
-- Retrieve total enrollment count by semester and year
SELECT t.semester, t.year, COUNT(*) AS total_enrollments
FROM Enrollments e
JOIN Time t ON e.time_id = t.time_id
GROUP BY t.semester, t.year;

-- Retrieve student enrollment details along with course information
SELECT s.student_name, c.course_name, e.grade
FROM Enrollments e
JOIN Students s ON e.student_id = s.student_id
JOIN Courses c ON e.course_id = c.course_id;

-- Retrieve the average grade for each course
SELECT c.course_name, AVG(e.grade) AS average_grade
FROM Enrollments e
JOIN Courses c ON e.course_id = c.course_id
GROUP BY c.course_name;


TOPIC: Performance Optimization and Querying
    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:
Utilize batch processing techniques to load data in bulk instead of individual row insertion.
      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.


In [ ]:
import time
import random
import threading
from multiprocessing import Pool

# Function to simulate data loading for a single row
def load_data(row):
    # Simulate data loading process
    time.sleep(random.uniform(0.1, 0.5))

# Function to load data in batches using multi-threading
def load_data_threading(data, batch_size):
    threads = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        thread = threading.Thread(target=load_data, args=(batch,))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

# Function to load data in parallel using multiprocessing
def load_data_multiprocessing(data, batch_size):
    pool = Pool()
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        pool.map(load_data, batch)

    pool.close()
    pool.join()

# Main script
if __name__ == '__main__':
    # Generate sample data
    data = [row for row in range(1000)]

    # Define batch size and number of iterations
    batch_size = 100
    num_iterations = 10

    # Measure time taken for loading data before optimizations
    start_time = time.time()
    for _ in range(num_iterations):
        load_data(data)
    end_time = time.time()
    print(f"Time taken before optimizations: {end_time - start_time} seconds")

    # Measure time taken for loading data with multi-threading
    start_time = time.time()
    for _ in range(num_iterations):
        load_data_threading(data, batch_size)
    end_time = time.time()
    print(f"Time taken with multi-threading: {end_time - start_time} seconds")

    # Measure time taken for loading data with multiprocessing
    start_time = time.time()
    for _ in range(num_iterations):
        load_data_multiprocessing(data, batch_size)
    end_time = time.time()
    print(f"Time taken with multiprocessing: {end_time - start_time} seconds")
